## Sentiment Analysis
### We will train the model using IMDB Reviews and predict whether it is a bad or good review
#### This can be done by using Embeddings: Embedding is a layer where it has its own weights defined and when the words or paragraphs are passed, it will seggregate and push those into respective cluster group.
##### Tips: We have to convert the labels from tensors and then into lists ansd then into numpy arrays
##### Tip: We have to convert the sentences from tensors to numpy.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

Shuffling and writing examples to /home/jupyter/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB28MMM/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/jupyter/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB28MMM/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/jupyter/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB28MMM/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/jupyter/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
training_data,testing_data=imdb['train'],imdb['test']

In [4]:
training_data

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [180]:
type(training_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [6]:
training_list=list(training_data)

In [16]:
len(training_list)

25000

In [23]:
training_list[4598][1]

<tf.Tensor: shape=(), dtype=int64, numpy=1>

### Now we have got the values now. Lets create Training_categories and training labels

In [150]:
training_category=[]
training_label=[]

In [151]:
for cat,label in training_data:
    training_category.append(str(cat.numpy()))
    training_label.append(label.numpy())

In [152]:
len(training_category)

25000

In [153]:
#testing_sentences[:3]

In [154]:
training_label[:5]

[0, 0, 0, 1, 1]

### Lets do the same with testing

In [155]:
test_category=[]
test_label=[]
for cat,label in testing_data:
    test_category.append(str(cat.numpy()))
    test_label.append(label.numpy())

In [156]:
test_label[:3]

[1, 1, 0]

### Now we have category into string but labels in list. Lets convert them into numpy arrays

In [157]:
training_label_numpy=np.array(training_label)
test_label_numpy=np.array(test_label)

In [158]:
training_label_numpy=training_label_numpy.reshape(-1,1)
test_label_numpy=test_label_numpy.reshape(-1,1)

In [159]:
training_label_numpy.shape

(25000, 1)

In [160]:
training_label_numpy[:5]

array([[0],
       [0],
       [0],
       [1],
       [1]])

In [179]:
test_category[0]

'b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel\'s absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven\'t laughed this hard since I saw THE FULL MONTY. (And, even then, I don\'t think I laughed quite this hard... So to speak.) Tukel\'s talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there\'s none of the over-the-top scenery chewing one might\'ve expected from a film like this). DING-A-LING-LESS is a film whose time has come."'

## Its Show time !!

In [161]:
from tensorflow import keras

##### Let us load some parameters

In [162]:
vocab_size=10000
max_length=120
embedded_dim=16
oov_text='<OOV>'
padding='post'

#### Tokenization --->

In [163]:
tokenization=keras.preprocessing.text.Tokenizer(num_words=vocab_size,oov_token='<OOV>')

In [164]:
tokenization.fit_on_texts(training_category)

In [165]:
word_index=tokenization.word_index

In [166]:
word_index['surprise']

872

### Texts to sequences

In [167]:
train_sequence=tokenization.texts_to_sequences(training_category)
test_sequence=tokenization.texts_to_sequences(test_category)

In [178]:
#test_sequence[0]

### padding of sequences

In [169]:
train_pad_sequences= keras.preprocessing.sequence.pad_sequences(train_sequence,maxlen=max_length,
                                                                padding=padding)
test_pad_sequences=keras.preprocessing.sequence.pad_sequences(test_sequence,maxlen=max_length,
                                                             padding=padding)

#### Layer creation

In [170]:
from tensorflow.keras.layers import Embedding,Flatten,Dense

In [181]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embedded_dim,input_length=max_length))
model.add(Flatten())
model.add(Dense(units=6,activation=tf.nn.relu))
model.add(Dense(units=1,activation=tf.nn.sigmoid))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [182]:
epochs=10
model.fit(train_pad_sequences,training_label_numpy,epochs=epochs,validation_data=(test_pad_sequences,test_label_numpy))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 10s 416us/sample - loss: 0.4797 - accuracy: 0.7438 - val_loss: 0.3412 - val_accuracy: 0.8482
Epoch 2/10
25000/25000 [==============================] - 10s 387us/sample - loss: 0.2189 - accuracy: 0.9173 - val_loss: 0.3415 - val_accuracy: 0.8512
Epoch 3/10
25000/25000 [==============================] - 9s 367us/sample - loss: 0.0903 - accuracy: 0.9771 - val_loss: 0.4148 - val_accuracy: 0.8425
Epoch 4/10
25000/25000 [==============================] - 10s 388us/sample - loss: 0.0279 - accuracy: 0.9958 - val_loss: 0.4884 - val_accuracy: 0.8387
Epoch 5/10
25000/25000 [==============================] - 10s 412us/sample - loss: 0.0105 - accuracy: 0.9988 - val_loss: 0.5409 - val_accuracy: 0.8392
Epoch 6/10
25000/25000 [==============================] - 9s 375us/sample - loss: 0.0056 - accuracy: 0.9993 - val_loss: 0.5932 - val_accuracy: 0.8378
Epoch 7/10
25000/25000 [======================

### Lets create another model but not with Flatten layer but with Global Average Pooling1D 

In [183]:
from tensorflow.keras.layers import GlobalAveragePooling1D

In [186]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embedded_dim,input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=5,activation=tf.nn.relu))
model.add(Dense(units=1,activation=tf.nn.sigmoid))
model.compile(optimizer='adam',loss=keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 85        
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 6         
Total params: 160,091
Trainable params: 160,091
Non-trainable params: 0
_________________________________________________________________


In [187]:
model.fit(train_pad_sequences,training_label_numpy,epochs=epochs,validation_data=(test_pad_sequences,test_label_numpy))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 11s 422us/sample - loss: 0.6855 - accuracy: 0.5238 - val_loss: 0.6454 - val_accuracy: 0.6817
Epoch 2/10
25000/25000 [==============================] - 11s 456us/sample - loss: 0.5971 - accuracy: 0.8207 - val_loss: 0.5839 - val_accuracy: 0.8396
Epoch 3/10
25000/25000 [==============================] - 10s 385us/sample - loss: 0.5666 - accuracy: 0.8746 - val_loss: 0.5759 - val_accuracy: 0.8393
Epoch 4/10
25000/25000 [==============================] - 10s 407us/sample - loss: 0.5555 - accuracy: 0.8948 - val_loss: 0.5727 - val_accuracy: 0.8490
Epoch 5/10
25000/25000 [==============================] - 7s 295us/sample - loss: 0.5477 - accuracy: 0.9121 - val_loss: 0.5716 - val_accuracy: 0.8562
Epoch 6/10
25000/25000 [==============================] - 10s 397us/sample - loss: 0.5418 - accuracy: 0.9238 - val_loss: 0.5714 - val_accuracy: 0.8522
Epoch 7/10
25000/25000 [=====================

### We have a nice model. Now if we want to visualise the word embedding, Below are the steps to be followed
#### 1. Take out the weights of the embedding layer via model.layer.weights
#### 2. There you will find the list of weights for every word present
#### 3. In the above problem we have 10000 words and we will find weights for all the 10k words.
#### 4. We will create two files where we ill store weights in one file and respectve words for those weights in another.
#### 5. As we have word_index but as a values we have numbers. We need to reverse the dictionary and then assign the weights to the values.
#### 6. We will download them and upload in the projector.tensorflow website to view the cluster of embeddings

In [205]:
### WE will reverse the word index first
reverse_word_index=dict([(i,w) for (w,i) in word_index.items()])

In [210]:
reverse_word_index[9999]

'socialist'

### Now we will create two files with format as tsv
### we wil load the weights and words

In [222]:
### We will pick  the weights first
layer=model.layers[0]
weights_list=layer.get_weights()
arr_weights=weights_list[0]

In [224]:
arr_weights[1]

array([ 0.01676033, -0.10503941, -0.28673616, -0.0178824 ,  0.0012398 ,
        0.27213126, -0.02414835, -0.20349813,  0.04617301,  0.09460002,
        0.03749573,  0.20931901, -0.01966789,  0.02536215,  0.03766212,
        0.10380462], dtype=float32)

In [211]:
import io

In [212]:
word_file=io.open('word_file.tsv','w',encoding='utf-8')
vector_file=io.open('vector_file.tsv','w',encoding='utf-8')


In [243]:
for i in range(vocab_size):
    ### As we reversed the dictionary, we wond have value of 0 as it is not index it is value.
    ### So the values starts from 1.
    word_file.write(reverse_word_index[i+1])
    ### As it these are weights we have to convert them to string and then 
    ### append with tab seperator for every weight and then we ill append new line for next word weight
    vector_file.write('\t'.join([str(x) for x in arr_weights[i]]) + '\n')


In [244]:
word_file.close()
vector_file.close()

In [237]:
'\t'.join([str(x) for x in arr_weights[1]]) + '\n'

'0.016760325\t-0.10503941\t-0.28673616\t-0.017882396\t0.0012398048\t0.27213126\t-0.024148349\t-0.20349813\t0.046173014\t0.09460002\t0.03749573\t0.20931901\t-0.019667894\t0.025362145\t0.03766212\t0.10380462\n'

'[ 0.00920381 -0.01133953 -0.1318373  -0.02579551  0.0007072   0.10600454\n -0.05109627 -0.06589282  0.00291154  0.03623795 -0.02652154  0.05861582\n  0.03622925 -0.01015301 -0.04672671  0.01281283]\t[ 0.01676033 -0.10503941 -0.28673616 -0.0178824   0.0012398   0.27213126\n -0.02414835 -0.20349813  0.04617301  0.09460002  0.03749573  0.20931901\n -0.01966789  0.02536215  0.03766212  0.10380462]\t[ 0.14546916  0.03354151 -0.19611074 -0.14200978  0.11267764  0.24479473\n  0.00029444 -0.06537913 -0.04477925 -0.03161895 -0.09601128  0.13920079\n  0.0627014  -0.11746637 -0.07766797  0.06718884]\n'

##### https://projector.tensorflow.org/ : Here we will upload both the files and visualize it

In [ ]:
z=['hey','hello','hi']
x=[]
for i in z:
   x.append() 